Import and read in the data.
'PICO-annotations' should be in the same directory as 'dawid_skene_pico'.
This will read in the population part of the 'PICO-annotations' dataset.
To run on other parts, change lines 9 to 13 in pico.py.

In [4]:
import pico, hmm, util

In [5]:
pico.read_ann()

32535

In [6]:
(cd, gold, list_wid, features, labels) = pico.main()

11363044


In [7]:
n = 2
m = len(features) + 1
hc = hmm.HMM_crowd(n, m, cd, features, labels, n_workers=len(list_wid), ne = 0, smooth = 1e-3)

Run Dawid-Skenne

In [8]:
hc.init(init_type='dw', wm_rep='cm', dw_em = 5, wm_smooth=0.001)

In [9]:
inv_l = {v:k for (k,v) in labels.items()}

Assemble the results

In [10]:
def list_word_spans(x):
    res = []
    i = 0
    while i < len(x):
        if (i < len(x)) and (x[i] == 0): i += 1; continue
        start = i
        while (i < len(x)) and (x[i] == 1): i += 1
        end = i - 1
        res.append( (start, end))
    return res

def get_pid(ins):
    s = inv_l[ins.label]
    a = map(int,s.split('_'))
    return a[0]

def get_start(ins):
    s = inv_l[ins.label]
    a = map(int,s.split('_'))
    return a[1]

def get_end(ins):
    s = inv_l[ins.label]
    a = map(int,s.split('_'))
    return a[2]


In [11]:
a = []
for s, r in zip(hc.data.sentences, hc.d.res):
    if len(s) == 0: continue
    pid = get_pid(s[0])
    spans = list_word_spans(r)
    for l,r in spans:
        start = get_start(s[l])
        end = get_end(s[r])
        a.append((pid, start, end, pico.dic_pid[str(pid)][start:end]))

The results is in 'a'.
Each element of 'a' is a tuple of (PUBMED ID, start of span, end of span, the span text) 

In [12]:
a

[(23549581,
  66,
  131,
  'patients with chemotherapy-induced painful peripheral neuropathy:'),
 (23549581,
  536,
  659,
  '231 patients who were 25 years or older being treated at community and academic settings between April 2008 and March 2011.'),
 (23549581,
  870,
  1158,
  'Eligibility required that patients have grade 1 or higher sensory neuropathy according to the NCI Common Terminology Criteria for Adverse Events and at least 4 on a scale of 0 to 10, representing average chemotherapy-induced pain, after paclitaxel, other taxane, or oxaliplatin treatment.'),
 (7808649, 133, 159, 'post-cholecystectomy pain]'),
 (7808649,
  332,
  398,
  'patients suffering from post-operative pain after cholecystectomy.'),
 (7808649, 488, 535, '99 patients with severe pain following surgery.'),
 (7808649, 1646, 1659, 'after surgery'),
 (3296982, 28, 54, 'type II diabetes mellitus.'),
 (3296982,
  88,
  171,
  'patients with type II diabetes mellitus for whom insulin therapy is most beneficial'

Now run the HMM-Crowd model

In [13]:
hc.vb = [0.1, 0.1]
hc.em(1)

hmm.py:400: RuntimeWarning: divide by zero encountered in log
  ll = np.log( np.sum(alpha[t-1]) )


In [14]:
hc.mls()

hmm.py:82: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  res[t] = b[t + 1, res[t + 1]]


In [15]:
b = []
for s, r in zip(hc.data.sentences, hc.res):
    if len(s) == 0: continue
    pid = get_pid(s[0])
    spans = list_word_spans(r)
    for l,r in spans:
        start = get_start(s[l])
        end = get_end(s[r])
        b.append((pid, start, end, pico.dic_pid[str(pid)][start:end]))

The results are in 'b'. Similar to 'a', Each element of 'b' is a tuple of (PUBMED ID, start of span, end of span, the span text)

In [16]:
b

[(23549581,
  66,
  131,
  'patients with chemotherapy-induced painful peripheral neuropathy:'),
 (23549581,
  451,
  512,
  '8 National Cancer Institute (NCI)-funded cooperative research'),
 (23549581,
  527,
  659,
  'enrolled 231 patients who were 25 years or older being treated at community and academic settings between April 2008 and March 2011.'),
 (23549581,
  870,
  1158,
  'Eligibility required that patients have grade 1 or higher sensory neuropathy according to the NCI Common Terminology Criteria for Adverse Events and at least 4 on a scale of 0 to 10, representing average chemotherapy-induced pain, after paclitaxel, other taxane, or oxaliplatin treatment.'),
 (7808649, 133, 159, 'post-cholecystectomy pain]'),
 (7808649,
  332,
  398,
  'patients suffering from post-operative pain after cholecystectomy.'),
 (7808649, 488, 535, '99 patients with severe pain following surgery.'),
 (7808649, 1646, 1659, 'after surgery'),
 (3296982, 28, 54, 'type II diabetes mellitus.'),
 (329698